In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

movies = pd.read_csv('../../data/ml-1m/movies.dat', sep='::', header=None, names=['MovieID', 'Title', 'Genres'], engine='python', encoding='latin-1')
ratings = pd.read_csv('../../data/ml-1m/ratings.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python', encoding='latin-1')

user_item_matrix = ratings.pivot(index='UserID', columns='MovieID', values='Rating')
user_item_matrix = user_item_matrix.fillna(0)

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def get_user_recommendations(user_id, user_item_matrix, user_similarity_df, n_recommendations=10):
    sim_scores = user_similarity_df[user_id]
    sim_users = sim_scores.sort_values(ascending=False).index[1:]
    user_ratings = user_item_matrix.loc[user_id]
    weighted_ratings = pd.Series(0, index=user_item_matrix.columns)
    
    sim_sum = 0
    for sim_user in sim_users:
        sim_score = sim_scores[sim_user]
        sim_ratings = user_item_matrix.loc[sim_user]
        weighted_ratings += sim_ratings * sim_score
        sim_sum += sim_score
        if sim_sum > 1:
            break

    weighted_ratings = weighted_ratings / sim_sum
    weighted_ratings = weighted_ratings[user_ratings == 0]
    recommendations = weighted_ratings.sort_values(ascending=False).head(n_recommendations)
    recommended_movie_ids = recommendations.index
    
    return movies[movies['MovieID'].isin(recommended_movie_ids)][['Title', 'Genres']]

print(get_user_recommendations(1, user_item_matrix, user_similarity_df))


                                 Title  \
589   Silence of the Lambs, The (1991)   
592                   Pinocchio (1940)   
1575          L.A. Confidential (1997)   
2009           Jungle Book, The (1967)   
2011         Lady and the Tramp (1955)   
2012        Little Mermaid, The (1989)   
2016             101 Dalmatians (1961)   
2027            Sleeping Beauty (1959)   
2068            Charlotte's Web (1973)   
2789            American Beauty (1999)   

                                           Genres  
589                                Drama|Thriller  
592                          Animation|Children's  
1575             Crime|Film-Noir|Mystery|Thriller  
2009          Animation|Children's|Comedy|Musical  
2011  Animation|Children's|Comedy|Musical|Romance  
2012  Animation|Children's|Comedy|Musical|Romance  
2016                         Animation|Children's  
2027                 Animation|Children's|Musical  
2068                         Animation|Children's  
2789             